In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2022
DAY = int('07')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [6]:
lines, groups = linesplit("""
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
""".strip())

In [3]:
lines, groups = linesplit(raw_input, verbose=True)

1022 lines
1 groups
>>>
$ cd /
$ ls
dir bzcg
...


In [7]:
lines

['$ cd /',
 '$ ls',
 'dir a',
 '14848514 b.txt',
 '8504156 c.dat',
 'dir d',
 '$ cd a',
 '$ ls',
 'dir e',
 '29116 f',
 '2557 g',
 '62596 h.lst',
 '$ cd e',
 '$ ls',
 '584 i',
 '$ cd ..',
 '$ cd ..',
 '$ cd d',
 '$ ls',
 '4060174 j',
 '8033020 d.log',
 '5626152 d.ext',
 '7214296 k']

In [12]:
a = []
b = []

for line in lines:
  if m := re.findall(r'^\$', line):
    if b:
      a.append(b)
      b = []
    b.append(line)
  else:
    b.append(line)

if b:
  a.append(b)
a

[['$ cd /'],
 ['$ ls', 'dir a', '14848514 b.txt', '8504156 c.dat', 'dir d'],
 ['$ cd a'],
 ['$ ls', 'dir e', '29116 f', '2557 g', '62596 h.lst'],
 ['$ cd e'],
 ['$ ls', '584 i'],
 ['$ cd ..'],
 ['$ cd ..'],
 ['$ cd d'],
 ['$ ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']]

In [13]:
exes = a

In [14]:
d = {}
stack = [d]
for exe in exes:
  comm = exe[0]
  reses = exe[1:]
  # print(comm)
  
  if m := re.findall(r'^\$ cd (.*)', comm):
    cd_to = m[0]
    # print(m[0])
    
    if cd_to == '/':
      stack = [d]
    elif cd_to == '..':
      stack.pop()
      if not stack:
        stack = [d]
    else:
      newd = {}
      stack[-1][cd_to] = newd
      stack.append(newd)
    
    
    
  elif m := re.findall(r'^\$ ls', comm):
    print(exe)
    for res in reses:
      if m := re.findall(r'^dir (.*)', res):
        dirname = m[0]
      elif m := re.findall(r'^(\d+) (.*)', res):
        size = int(m[0][0])
        name = m[0][1]
        stack[-1][name] = size
        
        
d

['$ ls', 'dir a', '14848514 b.txt', '8504156 c.dat', 'dir d']
['$ ls', 'dir e', '29116 f', '2557 g', '62596 h.lst']
['$ ls', '584 i']
['$ ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']


{'b.txt': 14848514,
 'c.dat': 8504156,
 'a': {'f': 29116, 'g': 2557, 'h.lst': 62596, 'e': {'i': 584}},
 'd': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}

In [17]:
all_sizes = []

def calc(d):
  mysize = 0
  for i, j in d.items():
    if type(j) is int:
      mysize += j
    else:
      mysize += calc(j)
  global all_sizes
  all_sizes.append(mysize)
  return mysize
  
calc(d), all_sizes

(48381165, [584, 94853, 24933642, 48381165])

In [18]:
all_sizes = array(all_sizes)
sum(all_sizes[all_sizes <= 100000])

95437

In [35]:
submit1(_)

Submitting 1141028...


True

In [37]:
glob = all_sizes[-1]

In [44]:
have = 70000000 - all_sizes[-1]
need = 30000000 - have

In [45]:
sorted(all_sizes[all_sizes >= need])

[8278005, 9725484, 10715508, 11798147, 12367143, 14629551, 48008081]

In [55]:
submit2(_[0])

TypeError: 'bool' object is not subscriptable